# Agregace

- Základní agregace za pomocí metod agg() a aggregate(), kdy lze použít více agregačních metod najednou, viz https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.agg.html a https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.aggregate.html

- Kumulované agregace za pomocí metod cummax(), cummin(), cumsum() apod. viz např.  https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.cummax.html

- Klouzavé agregace za pomoci metody rolling(), viz https://pandas.pydata.org/pandas-https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rolling.html

### Shlukování dat za pomoci metody groupby()

- Metoda groupby() slouží ke shlukování dat, přičemž v případě DataFrame můžeme shlukovat buď řádky (axis 0) nebo sloupce (axis 1),viz https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html

### Transformace dat

- Transformace řádků na sloupce a opačně za pomoci metody transpose(), viz https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.transpose.html
- Transformace řádků a sloupců s unikátními hodnotami za pomoci metody pivot(), viz https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pivot.html
Pozn. Metoda nepodporuje agregace
- Transormace řádků a sloupců s agregací za pomoci metody pivot_table(), viz https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pivot_table.html



In [16]:
# Import potrebnych knihoven
import pandas as pd
import numpy as np


# Stazeni .csv souboru publikovaneho na github.com
url = 'https://raw.githubusercontent.com/petrrozkosny/pydata/main/pydata_data.csv'
df = pd.read_csv(url,delimiter = ';')
df['DATE'] = pd.to_datetime(df['DATE'])
df['ROK'] = df['DATE'].dt.year
df['MESIC'] = df['DATE'].dt.month

df = df.groupby(['ROK','MESIC']).agg({'PRCP':'sum'})

df.loc[(df.index.get_level_values('MESIC') == 1) & (df.index.get_level_values('ROK') == 2010)]



PRCP
ROK  MESIC       
2010 1      398.8

In [2]:
srazky = df.copy()
print(srazky.iloc[0:10,:].transpose())


                           0                    1                    2  \
STATION          GME00132274          GME00132274          GME00132274   
NAME                  PIDING               PIDING               PIDING   
DATE     2006-12-01 00:00:00  2006-12-02 00:00:00  2006-12-03 00:00:00   
PRCP                     0.0                  0.0                  0.0   
SNWD                     0.0                  0.0                  0.0   
TAVG                     NaN                  NaN                  NaN   
TMAX                     4.2                  1.8                  5.1   
TMIN                     1.0                 -1.7                 -4.6   

                           3                    4                    5  \
STATION          GME00132274          GME00132274          GME00132274   
NAME                  PIDING               PIDING               PIDING   
DATE     2006-12-04 00:00:00  2006-12-05 00:00:00  2006-12-06 00:00:00   
PRCP                    19.0         

In [3]:
srazky.iloc[0:10,:].pivot(index='DATE',columns=['NAME']).head()


STATION   PRCP   SNWD   TAVG   TMAX   TMIN
NAME             PIDING PIDING PIDING PIDING PIDING PIDING
DATE                                                      
2006-12-01  GME00132274    0.0    0.0    NaN    4.2    1.0
2006-12-02  GME00132274    0.0    0.0    NaN    1.8   -1.7
2006-12-03  GME00132274    0.0    0.0    NaN    5.1   -4.6
2006-12-04  GME00132274   19.0    0.0    NaN    8.3   -0.9
2006-12-05  GME00132274    0.0    0.0    NaN   12.0    0.9

In [4]:
srazky1 = df.copy()
srazky1['ROK'] = srazky1['DATE'].dt.year
srazky1['PRCP'] = srazky1['PRCP'].fillna(0)
srazky1 =srazky1.pivot_table(values='PRCP',index='NAME',columns=['ROK'],aggfunc='sum',dropna=True,fill_value='-')
print(srazky1.head())

ROK                 2000   2001   2002   2003   2004   2005   2006    2007  \
NAME                                                                         
GRAZ               755.3  534.3  872.1  660.8  899.5  890.0  754.1   826.4   
KOSICE             530.8  566.4  648.4  491.9  645.8  701.2  650.7   630.9   
MOSNOV             502.3  629.0  504.4  448.9  512.2  643.8  548.7   708.8   
PIDING                 -      -      -      -      -      -   35.8  1379.2   
PRAHA KLEMENTINUM  385.9  552.0  661.5  267.1  394.2   80.6      -       -   

ROK                  2008    2009  ...    2011    2012    2013    2014   2015  \
NAME                               ...                                          
GRAZ                950.6  1204.3  ...   458.7   618.2   647.8   898.7  756.1   
KOSICE              645.2   613.1  ...   524.1   548.9   616.3   697.0  535.2   
MOSNOV              696.7   710.5  ...   599.8   598.0   581.3   747.7  421.7   
PIDING             1139.8  1367.8  ...  1103.1  

In [5]:
srazky2 = df.copy()
srazky2['ROK'] = srazky2['DATE'].dt.year
srazky2['MESIC'] = srazky2['DATE'].dt.month
srazky2 = srazky2.pivot_table(values=['PRCP'],index=['ROK','MESIC'],columns= ['NAME'], aggfunc = 'sum')
srazky2 = srazky2.droplevel(0,axis=1)
print(srazky2.head())


NAME        GRAZ  KOSICE  MOSNOV  PIDING  PRAHA KLEMENTINUM  PRAHA LIBUS  \
ROK  MESIC                                                                 
2000 1       9.2    18.9    30.7     NaN               17.0         29.4   
     2       0.5    36.6    20.8     NaN               18.3         18.5   
     3      67.6    31.6    31.2     NaN               81.4         48.5   
     4      18.1    43.4    40.4     NaN                6.5          2.9   
     5      90.9    57.3    71.7     NaN               38.7         10.9   

NAME        RUZYNE  SALZBURG  TURANY  WIEN  
ROK  MESIC                                  
2000 1        24.7      51.2    34.0  43.7  
     2        23.9      66.1    22.6  35.1  
     3        42.3     154.2    15.5  80.4  
     4         7.4      30.5     5.1  11.1  
     5        29.1     115.3    33.4  50.4  


## Cvičení

### Cvičení zadání
1. Z dat dostupných pod touto URL vytvořte DataFrame s názvem df: 'https://raw.githubusercontent.com/petrrozkosny/pydata/main/pydata_data.csv'
2. Změňte datový typ sloupce DATE na datetime
3. Do nového sloupce ROK spočítejte rok ze sloupce DATE
4. Do nového sloupce MESIC spočítejte měsíc ze sloupce DATE
5. DataFrame df omezte na NAME == 'RUZYNE'
6. DataFrame df agregujte dle sloupců ROK,MESIC, agregujte sloupec PRCP, agregační funkce suma, sloupce nenastavujte jako index

7. DataFrame df prevedte (pivotujte) sloupec MESIC do sloupců, sloupec ROK na index (pozn. netřeba používate pivot_table)
8. Nahraďte NaN hodnoty hodnotou '--'